In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

%run ../notebook-init.py

# load R libraries                                                                                    
%R invisible(library(ggplot2))
%R invisible(library(fastcluster))
%R invisible(library(reshape))
%R invisible(library(reshape2))
%R invisible(library(gplots))
%R invisible(library(RSQLite))

#set up ggplot2 defaults                                                                              
%R theme_set(theme_gray(base_size=18))

%pylab inline
pylab.rcParams['figure.figsize'] = (20, 20)


Populating the interactive namespace from numpy and matplotlib


In [25]:
# Count proportion of reads classified by AC-Diamond Fast and Kraken

import gzip
import sys
import subprocess as sp
import numpy as np
import glob

def countAligned(cinfile):
    containingDir = '/'.join(cinfile.split('/')[:-1])
    base = cinfile.split('/')[-1]
    species = base.split('.')[1].split('-')[0].split('_')[0]
    
    if '.acdmndfast.' in base:
        root = base[15:]
        root = root[: root.index('.acdmndfast.tsv.gz')]
    elif '.dmndfast.' in base:
        root = base[15:]
        root = root[: root.index('.dmndfast.tsv.gz')]
    elif '.classified.' in base:
        root = base[5:]
        root = root[: root.index('.classified.tsv.gz')]
        
    linecountfile = containingDir + '/lines_in.' + root + '.fastq.gz'
    try:
        with open(cinfile) as c:
            C = int(c.read().strip())
        with open(linecountfile) as l:
            L = int(l.read().strip())
        P = C / (L / 4.0)
        return (species, P)
    except:
        return False

def mergeAligned(outputs):
    merged = {}
    for species, out in outputs:
        if species not in merged:
            merged[species] = []
        merged[species].append(out)
    return merged

    
def crunchAligned(merged):
    crunched = {}
    for sp, vals in merged.items():
        u = np.mean(vals)
        sd = np.std(vals)
        crunched[sp] = (u,sd)
    return crunched

    
def main(args):
    if len(args) == 0:
        print("Usage: unique-seq-id-files")
        return 1

    outputs = []
    for f in args:
        a = countAligned(f)
        if not a:
            continue
        outputs.append(a)
    
    merged = mergeAligned(outputs)
    crunched = crunchAligned(merged)
    for k,v in crunched.items():
        print("{} : {}".format(k,v))
    return crunched

        
print('AC-DIAMOND FAST')
files = glob.glob('../line-counts/unique-seq-ids*')
main(files)

print('KRAKEN')
files = glob.glob('../line-counts/c_in*')
foo = main(files)



AC-DIAMOND FAST
rumen : (0.56098696648721968, 0.037682186497172525)
filtered : (0.67655579737798133, 0.097130202536103938)
cowrumen : (0.36098364658512183, 0.037225112641710564)
gpigfecal : (0.15674018308756352, 0.021303337094552302)
human : (0.61353927357687854, 0.11797808672311676)
KRAKEN
rumen : (0.074350605978206363, 0.015339479815349865)
filtered : (0.22414082702759297, 0.090995852961512289)
cowrumen : (0.025342301232090612, 0.0031820706302615864)
gpigfecal : (0.037189487458815625, 0.015610710761141441)
human : (0.26936412296020734, 0.1689764696662209)


In [56]:
# Count the proportion of classified reads which were assigned to different kingdoms for AC-Diamond

import gzip
import sys
import subprocess as sp
import numpy as np
import pandas as pd

with open('../phylum2kingdom.txt') as p2k:
    phylumtokingdom = p2k.read().strip()


def countAligned(alignfile,p2k):
    base = alignfile.split('/')[-1]
    #species = base.split('.')[1].split('-')[0].split('_')[0]
    species = alignfile.split('/')[-1].split('.')[0].split('-')[0].split('_')[0]
    
    if 'sheep' in alignfile:
        species = 'sheep'
    if 'cow' in alignfile:
        species='cow'
    sind = alignfile.index('phylum')
    sfile = alignfile[:sind] + 'species' + alignfile[sind+6:]
    #print(sfile)
    
    out = {
        'Bacteria':0,  
        'Viruses':0,
        'Archaea':0,           
    }
    with gzip.open(alignfile) as af:
        for line in af:
            #print(line)
            line = line.split()
            if len(line) != 2 or 'taxa' in line[0]:
                continue
            k = p2k[line[0].strip()]
#            print("{} -> {} ({})".format(line[0].strip(),k,int(line[1])))
            out[k] = out[k] + int(line[1])
    try:
        cmd = "zcat {} | grep -i 'virus'".format(sfile)
        data = sp.check_output(cmd,shell=True)
        for line in data.split('\n'):
            line = line.split()
            if len(line) != 2 or 'Sample' in line[0]:
                continue
            k = 'Viruses'
            out[k] = out[k] + int(line[1])
    except:
        pass
    
    out['Bacteria'] = out['Bacteria'] - out['Viruses']
    N = sum([v for v in out.values()])
    for k,v in out.items():
        out[k] = 1000*1000*float(v)/N

    return (species, out)


def mergeAligned(outputs):
    merged = {}
    for species, out in outputs:
        for kind, val in out.items():
            if kind not in merged:
                merged[kind] = {}
            if species not in merged[kind]:
                merged[kind][species] = []
            merged[kind][species].append(val)
    return merged

def buffer(outputs):
    buff = {}
    for species, out in outputs:
        if species not in buff:
            buff[species] = []
        buff[species].append(out)

    for species,outs in buff.items():
        top = 0
        if len(outs) > len(top):
            top = len(outs)
        for out in outs:
            for _ in range(top - len(out)):
                out.append(0)
    

def crunchAligned(merged):
    crunched = {kind:{} for kind in merged.keys()}
    for kind, val in merged.items():
        for species, vals in val.items():
            u = np.mean(vals)
            sd = np.std(vals)
            crunched[kind][species] = (u,sd)
    return crunched

def prettyprint(crunched):
    print("\n--------------------------------------------------------\n")
    outline = "{:.1f} ({:.1f}) \t& {:.1f} ({:.1f}) \t& {:.1f} ({:.1f}) \t& {:.1f} ({:.1f}) \t\\\\"
    for row in crunched.itertuples():
        print(row)
        ar = []
        for v in row[1:]:
  #          print(v)
            ar.append(float(v[0]))
            ar.append(float(v[1]))
 #       print(ar)
        print(outline.format(*ar))
        
    
def main(args):
    if len(args) == 0:
        print("Usage: metaphlan-profile-files")
        return 1

    p2k = {}
    a = phylumtokingdom.split('\n')
    a = [v for v in a if len(v) > 0]
    for i in range(0,len(a),2):
        p2k[a[i].strip()] = a[i+1].strip()

#     for k,v in p2k.items():
#         print("{} : {}".format(k,v))
    
    outputs = []
    for f in args:
        a = countAligned(f,p2k)
        if not a:
            continue
        outputs.append(a)
    
    merged = mergeAligned(outputs)
    crunched = crunchAligned(merged)
    crunched = pd.DataFrame(crunched)
    print(crunched)
    #prettyprint(crunched)

files = glob.glob('../taxonomic-profiles/ac-diamond-fast/individual-profiles/*phylum*')
main(files)

                                  Archaea                        Bacteria  \
cow        (47687.8564864, 31782.5803761)  (952297.902674, 31782.4887042)   
filtered   (414.214000415, 176.144950445)  (999529.574194, 201.984696156)   
gpigfecal  (5947.09360352, 2946.97458962)  (994049.721574, 2948.38205831)   
human      (1528.03356317, 4403.62319599)  (998261.202815, 4459.28212911)   
rumen      (5103.37955674, 2955.45063704)  (984896.831432, 4439.35560583)   

                                  Viruses  
cow         (14.2408399156, 7.7811772283)  
filtered   (56.2118058899, 58.2316192273)  
gpigfecal   (3.18482296343, 2.0517725018)  
human      (210.763621407, 1013.55515629)  
rumen      (9999.78901145, 2692.39994652)  


In [58]:
# Count the proportion of classified reads which were assigned to different kingdoms for AC-Diamond

import gzip
import sys
import subprocess as sp
import numpy as np
import pandas as pd


def countAligned(alignfile):
    base = alignfile.split('/')[-1]
    tool = 'kraken'
    species = alignfile.split('/')[-1].split('.')[1].split('-')[0].split('_')[0]
    
    
    out = {
        'Bacteria':0,  
        'Viruses':0,
        'Archaea':0,           
    }
    with open(alignfile) as af:
        for line in af:
            line = line.split()
            if len(line) != 2 or 'taxa' in line[0]:
                continue
            k = line[0].strip().split('__')[1]
            out[k] = out[k] + int(line[1])
    
    N = sum([v for v in out.values()])
    if N == 0 :
        print(alignfile)
        return False
    for k,v in out.items():
        out[k] = 1000*1000*float(v)/N

    return (species, out)


def mergeAligned(outputs):
    merged = {}
    for species, out in outputs:
        for kind, val in out.items():
            if kind not in merged:
                merged[kind] = {}
            if species not in merged[kind]:
                merged[kind][species] = []
            merged[kind][species].append(val)
    return merged
    

def crunchAligned(merged):
    crunched = {kind:{} for kind in merged.keys()}
    for kind, val in merged.items():
        for species, vals in val.items():
            u = np.mean(vals)
            sd = np.std(vals)
            crunched[kind][species] = (u,sd)
    return crunched
        
    
def main(args):

    outputs = []
    for f in args:
        a = countAligned(f)
        if not a:
            continue
        outputs.append(a)
    
    merged = mergeAligned(outputs)
    crunched = crunchAligned(merged)
    crunched = pd.DataFrame(crunched)
    print(crunched)

files = glob.glob('../line-counts/kraken_phyla*')
main(files)

../line-counts/kraken_phyla.human-posterior_fornix-R056796.classified.tsv
                                  Archaea                        Bacteria  \
cowrumen   (62985.3027155, 30345.0069695)  (869397.718672, 34322.4963739)   
filtered   (368.994187329, 226.225373368)  (998713.041574, 475.982607408)   
gpigfecal  (6222.03133467, 3980.41843104)   (993118.46342, 4011.76636231)   
human      (5590.04867586, 17023.0934813)   (982380.95938, 26917.3489289)   
rumen      (13263.0803366, 6845.71852958)  (917390.322527, 23109.2876915)   
setA1                (14831.0834705, 0.0)            (985087.959438, 0.0)   
setB1                (69094.8236298, 0.0)            (930822.374725, 0.0)   
setB2                 (69088.446011, 0.0)            (930835.866591, 0.0)   

                                  Viruses  
cowrumen   (67616.9786129, 9585.21486643)  
filtered   (917.964239157, 309.533142928)  
gpigfecal  (659.505245315, 214.960768491)  
human      (12028.9919443, 23526.7720458)  
rumen      (